## Speech-to-Text mit Hugging Face Transformers (Whisper)

In diesem Workshop lernst du, wie du mithilfe von **Hugging Face**-Modellen (speziell Whisper) Sprachaufnahmen automatisiert transkribieren kannst. Wir verwenden dabei das Modell [openai/whisper-base](https://huggingface.co/openai/whisper-base).

Voraussetzungen:
- Installierte Pakete:
  - `transformers`
  - `torchaudio`
  - `tqdm`
  - `huggingface_hub`
- Eine Audio-Datei (z. B. `.mp3`, `.wav`) zum Testen

### 1. Import

In [10]:
import torch
import json
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from tqdm import tqdm

### 2. Dateipfade spezifizieren
- Wir definieren Ordner und Dateinamen (z. B. `audio_file_folder`, `audio_name`, `audio_format`, `whisper_folder`).
- Lege in `audio_file_folder` deine eigenen Audiodateien ab und ändere `audio_name` entsprechend, um deine eigenen Dateien zu transkribieren
- Damit wir die Ausgabe von Whisper nachher weiterverwenden können, speichern wir die Ausgabe später in einer .json File im `whisper_folder` ab. 

Wir benötigen eine **mp3** Datei als Eingabe für Whisper. Mit **ffmpeg** können wir Audiodateien von anderem Format konvertieren.

In [11]:
!ffmpeg -i ./audios/<audio_name>.m4a ./audios/<audio_name>.mp3

/bin/bash: line 1: audio_name: No such file or directory


In [12]:
# Spezifierung von Verzeichnissen und Dateinamen
audio_file_folder = "./audios/"
audio_name = "Sonic-Manus-Pitch"
audio_format = ".mp3"
whisper_folder = "./outputs/whisper/"

### 3. Device wählen (GPU oder CPU)
Wir wählen automatisch die GPU aus, wenn `torch.cuda.is_available()` den Wert `True` zurückgibt. Ansonsten fallen wir auf die CPU zurück. Zudem passen wir den **Datentyp** (`torch_dtype`) an, um auf der GPU mit `float16` rechnen zu können.


In [13]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

### 4. Laden des Modells und Prozessors

- **AutoModelForSpeechSeq2Seq**: Lädt ein Modell, das speziell für die Sprach-zu-Text-Aufgaben (Speech-to-Text) ausgelegt ist.
- **AutoProcessor**: Lädt den passenden Tokenizer und Audio-Feature-Extractor, um die Audiodatei in das richtige Input-Format für das Modell zu konvertieren.

Anschließend verschieben wir das Modell auf das gewählte `device`.

Das [Modell wird von Huggingface](https://huggingface.co/openai/whisper-large-v3-turbo) heruntergeladen. Dort findest du auch weitere interessante Informationen über das Model.

In [14]:
# Modell-ID von Whisper
# model_id = "openai/whisper-large-v3-turbo"
model_id = "openai/whisper-base"

# Modell laden
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, 
    torch_dtype=torch_dtype, 
    low_cpu_mem_usage=True, 
    use_safetensors=True,
)
model.to(device)

# Prozessor laden
processor = AutoProcessor.from_pretrained(model_id)


### 5. Festlegen der Sprache

Mit `forced_decoder_ids` legen wir fest, dass das Modell **Deutsch** transkribieren soll. 
Der Aufruf `processor.get_decoder_prompt_ids(language="de", task="transcribe")` erzeugt die IDs, die sicherstellen, dass das Modell deutschsprachigen Text als Output generiert.

Probiere gerne andere Sprachen aus und schaue wie das funktioniert passiert.


In [15]:
# Forced Decoder IDs für Deutsch (Transkription)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="de", task="transcribe")

### 6. Pipeline erstellen

Wir erstellen eine Pipeline vom Typ `"automatic-speech-recognition"`. Damit:
- wird das Modell und der Tokenizer verknüpft
- das Audio-Feature-Extrahieren übernimmt `feature_extractor`
- im `generate_kwargs` können wir die `forced_decoder_ids` übergeben
- mit `return_timestamps=True` erhalten wir ggf. Wort- oder Chunk-basierte Zeitstempel

So können wir direkt eine Audiodatei an `pipe(...)` übergeben, ohne manuell Tokenizing etc. durchzuführen.


In [16]:
# Pipeline einrichten
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
)

### 7. Transkribieren der Datei
- Das Modell transrkribiert die Datei
- `pipe(...)` gibt ein Dictionary mit Feldern zurück (z. B. `"text"` für die transkribierte Ausgabe, `"chunks"` für die Zeitstempel etc.).
- Mit `json.dumps(result, indent=4)` können wir das gesamte Dictionary (mit Timestamps, Token-Infos etc.) schön formatiert ausgeben.

In [17]:
result = pipe(f"{audio_file_folder}{audio_name}{audio_format}", generate_kwargs={"forced_decoder_ids": forced_decoder_ids})

# Ergebnis als JSON ausgeben
print(json.dumps(result, indent=4))

/usr/local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{
    "text": " Moin, was ist denn das Sonic Manus Projekt? Das Sonic Manus Projekt ist ein kairosiertes Transcription Tool. Das Tolle daran ist, dass es auch die unterschiedlichen Sprecher in einem Gespr\u00e4ch aufgeteilt bekommt. Okay, aber das kann Teams doch l\u00e4ngst. Er schon, aber Teams k\u00f6nnen daf\u00fcr einfach die unterschiedlichen Ger\u00e4te von den Benutzern verwenden und wir k\u00f6nnen das ganz von alleine. Genau, wir brauchen nur das Audio und zus\u00e4tzlich l\u00e4uft es bei uns lokal. Mika will ich haben.",
    "chunks": [
        {
            "timestamp": [
                0.0,
                3.2
            ],
            "text": " Moin, was ist denn das Sonic Manus Projekt?"
        },
        {
            "timestamp": [
                3.2,
                7.04
            ],
            "text": " Das Sonic Manus Projekt ist ein kairosiertes Transcription Tool."
        },
        {
            "timestamp": [
                7.04,
                13.0
 

### 8. Datei Speichern
- `result["text"]` enthält den transkribierten Text.
- Nun speichern wir das Ergebnis in einer `.json`-Datei.

In [18]:
# In Datei speichern
with open(f"{whisper_folder}{audio_name}.json", "w") as file:
    json.dump(result, file, indent=4)

## Zusammenfassung

In diesem Workshop haben wir:
1. Ein **Whisper-Modell** (openai/whisper-large-v3-turbo) geladen und konfiguriert.
2. Eine **Pipeline** für die automatische Spracherkennung eingerichtet.
3. **Forced Decoder IDs** benutzt, um Deutsch als Transkriptionssprache festzulegen.
4. Eine **Audiodatei** transkribiert und das Ergebnis als Text und JSON ausgegeben.

Damit hast du ein Grundgerüst, um eigene Audiodateien zu transkribieren oder Übersetzungen in andere Sprachen durchzuführen. 
- Passe je nach Bedarf die `forced_decoder_ids` an, um unterschiedliche Sprachen zu erzwingen.
- Experimentiere mit verschiedenen Whisper- oder anderen Speech-to-Text-Modellen von Hugging Face.
- Nutze die Timestamps für die Anzeige von Untertiteln oder Live-Transkriptionen.

Viel Erfolg!